In [1]:
import pandas as pd
import datetime as dt
import requests
from pprint import pprint

In [25]:
with open('D:\\lianz\\Desktop\\Python\\personal_projects\\finance_dashboard\\alpha_vantage_api.txt','r') as f:
    alpha_vantage_api = f.readlines()[0]

In [ ]:
#TODO: collect and load into SQL database

# Load API

In [38]:
def get_nba_info(endpoint: str, **kwargs):

    url = f"https://api-nba-v1.p.rapidapi.com/{endpoint}"

    headers = {
        "X-RapidAPI-Key": alpha_vantage_api,
        "X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=kwargs)

    r = response.json()

    return r

seasons = {'endpoint':'seasons'}
leagues = {'endpoint': 'leagues'}
games = {'endpoint': 'games', 'id': int, 'date': [str, 'YYYY-MM-DD'], 'live': str, 'league': str, 'season': int, 'team': int, 'h2h': str}
games_stats = {'endpoint': 'games/statistics', 'id': int}
teams = {'endpoint': 'teams', 'id': int, 'name': str, 'code':  str, 'league': str, 'conference': str, 'division': str, 'search': str}
team_stats = {'endpoint': 'teams/statistics', 'id': int, 'seaons': int}
players = {'endpoint': 'players', 'id': int, 'name': str, 'team': int, 'season': int, 'country': str, 'search': str}
player_stats = {'endpoint': 'players/statistics', 'id': int, 'game': int, 'team': int, 'season': int}
standings = {'endpoint': 'standings', 'league': str, 'season': int, 'team': int, 'conference': str, 'division': str,}

In [39]:
get_nba_info('players', search='james')
get_nba_info('players/statistics?season=2020&id=265')

{'get': 'players/statistics',
 'parameters': {'id': '265'},
 'errors': {'season': 'The Season field is required.'},
 'results': 0,
 'response': []}

In [6]:
import requests
import datetime

# Get the date for the previous night
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
date_string = yesterday.strftime("%Y%m%d")

# Build the URL for the scoreboard endpoint
scoreboard_url = f"https://stats.nba.com/stats/scoreboardv2?DayOffset=-1&LeagueID=00&gameDate={date_string}"

# Make a request to the scoreboard endpoint
response = requests.get(scoreboard_url)

# Get the GameIDs for all games played on the previous night
game_ids = []
for game in response.json()["games"]:
    game_ids.append(game["gameId"])

# Loop through each GameID and retrieve data for each game using the boxscoretraditionalv2 endpoint
for game_id in game_ids:
    boxscore_url = f"https://stats.nba.com/stats/boxscoretraditionalv2?EndPeriod=10&EndRange=28800&GameID={game_id}&RangeType=0&SeasonType=Regular+Season&StartPeriod=1&StartRange=0"
    response = requests.get(boxscore_url)
    for player in response.json()["resultSets"][0]["rowSet"]:
        # Extract the information for each player, such as points scored
        player_name = player[2]
        points_scored = player[21]
        print(f"{player_name}: {points_scored} points")
